# Load candidates

In [1]:
%load_ext autoreload
%autoreload 2


In [2]:
CACHE = '/home/anhphantq/vdb/cache_train'
import os
import pickle
from tqdm.notebook import tqdm
import numpy as np
import pandas as pd
import psutil

train_data = []

with open(CACHE + '/metadata', 'rb') as f:
  metadata = pickle.load(f)

num_frags = min(metadata['num_frags'], 10000)
max_session = metadata['max_session']

with tqdm(total= int (psutil.virtual_memory().total  / 1024  / 1024 / 1024), desc='RAM (GB)', position=0, leave= False) as rambar:

  for i in tqdm(range(0, num_frags)):

    train_data.append(pd.read_parquet(CACHE + f'/data_{i}.parquet'))
    rambar.n= int(psutil.virtual_memory().used / 1024  / 1024 / 1024)
    rambar.refresh()




RAM (GB):   0%|          | 0/125 [00:00<?, ?it/s]

  0%|          | 0/1801 [00:00<?, ?it/s]

In [3]:
train_data = pd.concat(train_data)

In [4]:
candidates = train_data
candidates.shape

(101668453, 112)

In [5]:
import pandas as pd
pd.set_option('display.max_columns', candidates.shape[1])

In [6]:
candidates

,user,item,num_clicks,num_carts,num_orders,item_degree,item_pr,recent_num_clicks,recent_num_carts,recent_num_orders,item_recent_degree,item_recent_pr,recent_day7_type0,recent_day7_type1,recent_day7_type2,recent_day6_type0,recent_day6_type1,recent_day6_type2,recent_day5_type0,recent_day5_type1,recent_day5_type2,recent_day4_type0,recent_day4_type1,recent_day4_type2,recent_day3_type0,recent_day3_type1,recent_day3_type2,recent_day2_type0,recent_day2_type1,recent_day2_type2,recent_day1_type0,recent_day1_type1,recent_day1_type2,item_glob_last_action,item_glob_first_action,item_glob_time_decay_sum,item_glob_time_decay_sum_click,item_glob_time_decay_sum_cart,item_glob_time_decay_sum_order,item_glob_item_glob_durability,qou_num_cosub_mean,qou_coclick_sub_coef_mean,qou_cocart_sub_coef_mean,qou_coorder_sub_coef_mean,qou_coclick_time_decay_mean,qou_cocart_time_decay_mean,qou_coorder_time_decay_mean,qou_num_appearance_mean,qou_num_in_k_most_recent_items_mean,qou_lincom_sub_coef_1_3_10_mean,qou_lincom_time_decay_1_3_10_mean,qou_lincom_sub_coef_1_10_3_mean,qou_lincom_time_decay_1_10_3_mean,qou_lincom_sub_coef_3_1_10_mean,qou_lincom_time_decay_3_1_10_mean,qou_lincom_sub_coef_3_10_1_mean,qou_lincom_time_decay_3_10_1_mean,qou_lincom_sub_coef_10_1_3_mean,qou_lincom_time_decay_10_1_3_mean,qou_lincom_sub_coef_10_3_1_mean,qou_lincom_time_decay_10_3_1_mean,qou_lincom_sub_coef_0.5_10_0.5_mean,qou_lincom_time_decay_0.5_10_0.5_mean,qou_lincom_sub_coef_10_0.5_0.5_mean,qou_lincom_time_decay_10_0.5_0.5_mean,qou_lincom_sub_coef_0.5_0.5_10_mean,qou_lincom_time_decay_0.5_0.5_10_mean,qou_num_cosub_sqrt_num_cousers,qou_coclick_sub_coef_sqrt_num_cousers,qou_cocart_sub_coef_sqrt_num_cousers,qou_coorder_sub_coef_sqrt_num_cousers,qou_coclick_time_decay_sqrt_num_cousers,qou_cocart_time_decay_sqrt_num_cousers,qou_coorder_time_decay_sqrt_num_cousers,qou_num_appearance_sqrt_num_cousers,qou_num_in_k_most_recent_items_sqrt_num_cousers,num_sub,consistency,num_actions,user_degree,user_pr,user_recent_degree,user_recent_pr,fitness,inter_clicks,inter_carts,inter_orders,inter_num_sub,inter_time_decay,inter_lts,inter_fts,inter_durability,inter_num_interacts,is_level1,user_clicks_cnt,user_carts_cnt,user_orders_cnt,user_total_actions_cnt,user_clicks_ratio,user_carts_ratio,user_orders_ratio,user_ss_ts_max,user_ss_ts_min,user_ss_ts_mean,item_item_cnt,item_item_clicks_cnt,item_item_carts_cnt,item_item_orders_cnt,item_item_total_actions_cnt,item_item_clicks_ratio,item_item_carts_ratio,item_item_orders_ratio
0,12201985.0,12907572.0,9990.0,287.0,51.0,6629.0,3.261024e-05,4704.0,156.0,32.0,3035.0,3.272604e-05,606.0,18.0,2.0,544.0,20.0,3.0,570.0,13.0,6.0,489.0,9.0,0.0,639.0,22.0,1.0,592.0,15.0,2.0,657.0,15.0,5.0,20.998276,0.001782,4649.395508,4500.714844,125.717499,22.963305,20.996492,0.562559,0.671306,0.881854,0.0,0.421352,0.512205,0.0,0.879936,0.746412,0.672127,0.418773,0.705886,0.437864,0.668556,0.418845,0.685268,0.428393,0.671379,0.421290,0.672556,0.421966,0.734613,0.452059,0.671473,0.421421,0.651170,0.404880,0.035504,0.002114,0.000061,0.0,0.000551,1.770033e-05,0.0,1.328157,0.003087,0.0,1.0,1.0,1.0,1.397941e-08,1.0,2.342916e-08,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.661502e+09,1.661502e+09,1.661502e+09,10272.0,9929.0,293.0,50.0,10272.0,0.966608,0.028524,0.004868
1,12201985.0,12907764.0,1541.0,39.0,8.0,1134.0,4.602560e-06,690.0,14.0,3.0,501.0,4.499182e-06,141.0,1.0,0.0,66.0,3.0,0.0,50.0,1.0,0.0,45.0,2.0,0.0,80.0,0.0,0.0,99.0,0.0,0.0,171.0,5.0,2.0,20.987835,0.013623,792.702698,770.866638,18.607828,3.228216,20.974213,0.415804,0.553407,0.000000,0.0,0.404785,0.000000,0.0,0.439968,0.373206,0.509913,0.366948,0.451536,0.318380,0.545885,0.398114,0.515318,0.371746,0.551874,0.403431,0.549676,0.401507,0.383899,0.264417,0.552746,0.404203,0.521742,0.376856,0.026242,0.001743,0.000000,0.0,0.000529,0.000000e+00,0.0,0.664078,0.001544,0.0,1.0,1.0,1.0,1.397941e-08,1.0,2.342916e-08,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.661502e+09,1.66

# Train params

In [7]:
from meow_utils import *
from catboost.utils import eval_metric

PARAMS = {
    'learning_rate': 0.5,
    'iterations': 1000,
    'early_stopping_rounds' : 20,
    'max_bin': 256,
    'depth': 7,
    'eval_metric': 'RecallAt:top=20;border=0.5',
    'use_best_model': True,
    'thread_count': -1,
}
N_SPLITS = 5
NUM_SAMPLES = 3
MODEL_PATH = '/home/anhphantq/meowmeow'
VERSION = '8'

# Train for each type

In [8]:
from meow_utils import get_len_group

In [9]:
import pandas as pd 
import numpy as np
tar = pd.read_parquet('/home/anhphantq/otto/splitted_data/test_labels.parquet')
tar = tar.loc[(tar['session'] <= candidates['user'].max()) & (tar['session'] >= candidates['user'].min()) ]
aids = tar.ground_truth.explode().astype('int32').rename('item')
tar = tar[['session', 'type']].rename({'session':'user'},axis=1)
tar = tar.merge(aids, left_index=True, right_index=True, how='left')
tar['type'] = tar['type'].apply(lambda x: 0 if x == 'clicks' else 1 if x == 'carts' else 2)
tar['label'] = 1
tar['item'] = tar['item'] + max_session
tar['user_type'] = tar['user'] * 10 + tar['type']

In [10]:
labels = []
users = candidates.user.values
items = candidates.item.values
from tqdm.notebook import tqdm
for i in tqdm(range(3)):
  tar_small = tar[tar['type'] == i].sort_values(['user', 'item']).set_index(['user', 'item'])
  
  tar_small_index = set(tar_small.index.tolist())
  label = []

  for i in tqdm(range(candidates.shape[0]), leave = False):
    
    label.append(1 if (users[i], items[i]) in tar_small_index else 0)
  
  labels.append(label)

del tar, tar_small_index
  



  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/101668453 [00:00<?, ?it/s]

  0%|          | 0/101668453 [00:00<?, ?it/s]

  0%|          | 0/101668453 [00:00<?, ?it/s]

In [11]:
labels = np.array(labels)

In [12]:
from sklearn.model_selection import GroupKFold
import os
import matplotlib.pyplot as plt
import shutil
from meow_utils import plot_importance
from catboost import CatBoostRanker, Pool
import seaborn as sns
sns.set()
%matplotlib inline
def train(t, id_type, draw_importance = False):
  global candidates, labels
  name = f'/{t}_only_'
  try:
    shutil.rmtree(MODEL_PATH + name + VERSION)
  except:
    pass
  else:
    print('remove old version')
  fig, axs = plt.subplots(1, 3, figsize = (300, 30))
  skf = GroupKFold(n_splits=N_SPLITS)
  
  
  X_all = candidates.iloc[:, 2 :]
  y_all = labels[id_type]
  print('Building pool...')
  data_all = Pool(X_all, y_all, thread_count = os.cpu_count())
  print('Pool builed!!!!!!')
  for fold,(train_idx, valid_idx) in enumerate(skf.split(candidates, labels[id_type], groups=candidates['user'] )):
      neg_index = np.ravel(np.argwhere(labels[id_type][train_idx] == 0))
      pos_index = np.ravel(np.argwhere(labels[id_type][train_idx] == 1))
      #downsample many times
      for sample in range(NUM_SAMPLES):
        print(f'FOLD: {fold} Sample: {sample}')
        n_index = np.random.permutation(neg_index)[:len(pos_index) * 5 * (1 + sample)]
        index = np.union1d(n_index, pos_index)
        
        data_train = data_all.slice(train_idx).slice(index)
        data_train.set_group_id(candidates['user'].values.astype(int)[train_idx][index])
        data_valid = data_all.slice(valid_idx)
        data_valid.set_group_id(candidates['user'].values.astype(int)[valid_idx])

        params = {'thread_count' : os.cpu_count(),
        **PARAMS}
        model = CatBoostRanker(**params)
        
        model.fit(data_train, eval_set = data_valid)
        
        try:
          os.mkdir(MODEL_PATH + name + VERSION)
        except:
          pass
        if fold < 5 and sample == 0 and draw_importance:
          plot_importance(X_all.columns.tolist(), model.get_feature_importance(data = data_valid, type = 'LossFunctionChange'), ax = axs[fold])
          axs[fold].bar_label(axs[fold].containers[0])
        model.save_model(MODEL_PATH + name + VERSION + f'/meowmewo_fold{fold}_sample{sample}')
  


In [13]:
for id_type, t in enumerate(['clicks', 'carts', 'orders']):
  train(t, id_type)

Building pool...
Pool builed!!!!!!
FOLD: 0 Sample: 0
0:	learn: 1.0000000	test: 0.8940696	best: 0.8940696 (0)	total: 1.31s	remaining: 21m 54s
1:	learn: 1.0000000	test: 0.9074809	best: 0.9074809 (1)	total: 2.4s	remaining: 19m 58s
2:	learn: 1.0000000	test: 0.9112561	best: 0.9112561 (2)	total: 3.52s	remaining: 19m 28s
3:	learn: 1.0000000	test: 0.9156816	best: 0.9156816 (3)	total: 4.94s	remaining: 20m 29s
4:	learn: 1.0000000	test: 0.9176485	best: 0.9176485 (4)	total: 6.41s	remaining: 21m 15s
5:	learn: 1.0000000	test: 0.9185209	best: 0.9185209 (5)	total: 7.71s	remaining: 21m 17s
6:	learn: 1.0000000	test: 0.9189769	best: 0.9189769 (6)	total: 9.03s	remaining: 21m 21s
7:	learn: 1.0000000	test: 0.9199722	best: 0.9199722 (7)	total: 10.5s	remaining: 21m 41s
8:	learn: 1.0000000	test: 0.9205671	best: 0.9205671 (8)	total: 11.9s	remaining: 21m 48s
9:	learn: 1.0000000	test: 0.9210548	best: 0.9210548 (9)	total: 13.2s	remaining: 21m 47s
10:	learn: 1.0000000	test: 0.9215307	best: 0.9215307 (10)	total: 14.

Training has stopped (degenerate solution on iteration 27, probably too small l2-regularization, try to increase it)



bestTest = 0.9970498309
bestIteration = 18

Shrink model to first 19 iterations.
FOLD: 1 Sample: 1
0:	learn: 0.9999973	test: 0.9946713	best: 0.9946713 (0)	total: 637ms	remaining: 10m 35s
1:	learn: 0.9999973	test: 0.9961380	best: 0.9961380 (1)	total: 1.21s	remaining: 10m 2s
2:	learn: 0.9999973	test: 0.9965105	best: 0.9965105 (2)	total: 1.78s	remaining: 9m 52s
3:	learn: 0.9999973	test: 0.9966150	best: 0.9966150 (3)	total: 2.38s	remaining: 9m 51s
4:	learn: 0.9999973	test: 0.9967498	best: 0.9967498 (4)	total: 3.02s	remaining: 10m
5:	learn: 0.9999973	test: 0.9967914	best: 0.9967914 (5)	total: 3.71s	remaining: 10m 13s
6:	learn: 0.9999973	test: 0.9969010	best: 0.9969010 (6)	total: 4.33s	remaining: 10m 14s
7:	learn: 0.9999973	test: 0.9968603	best: 0.9969010 (6)	total: 4.93s	remaining: 10m 10s
8:	learn: 0.9999973	test: 0.9969215	best: 0.9969215 (8)	total: 5.6s	remaining: 10m 16s
9:	learn: 0.9999973	test: 0.9969871	best: 0.9969871 (9)	total: 6.27s	remaining: 10m 21s
10:	learn: 0.9999973	test: 0

Training has stopped (degenerate solution on iteration 21, probably too small l2-regularization, try to increase it)



bestTest = 0.9971782472
bestIteration = 16

Shrink model to first 17 iterations.
FOLD: 2 Sample: 2
0:	learn: 0.9999978	test: 0.9944649	best: 0.9944649 (0)	total: 747ms	remaining: 12m 26s
1:	learn: 0.9999978	test: 0.9959969	best: 0.9959969 (1)	total: 1.44s	remaining: 12m
2:	learn: 0.9999978	test: 0.9964550	best: 0.9964550 (2)	total: 2.14s	remaining: 11m 51s
3:	learn: 0.9999978	test: 0.9966163	best: 0.9966163 (3)	total: 2.87s	remaining: 11m 54s
4:	learn: 0.9999978	test: 0.9968153	best: 0.9968153 (4)	total: 3.69s	remaining: 12m 13s
5:	learn: 0.9999978	test: 0.9969491	best: 0.9969491 (5)	total: 4.49s	remaining: 12m 24s
6:	learn: 0.9999978	test: 0.9970483	best: 0.9970483 (6)	total: 5.32s	remaining: 12m 34s
7:	learn: 0.9999978	test: 0.9970410	best: 0.9970483 (6)	total: 6.13s	remaining: 12m 39s
8:	learn: 0.9999978	test: 0.9970933	best: 0.9970933 (8)	total: 6.98s	remaining: 12m 48s
9:	learn: 0.9999978	test: 0.9971218	best: 0.9971218 (9)	total: 7.8s	remaining: 12m 52s
10:	learn: 0.9999978	test

Training has stopped (degenerate solution on iteration 34, probably too small l2-regularization, try to increase it)



bestTest = 0.9970262563
bestIteration = 32

Shrink model to first 33 iterations.
FOLD: 4 Sample: 0
0:	learn: 0.9999948	test: 0.9948532	best: 0.9948532 (0)	total: 473ms	remaining: 7m 52s
1:	learn: 0.9999948	test: 0.9960668	best: 0.9960668 (1)	total: 917ms	remaining: 7m 37s
2:	learn: 0.9999948	test: 0.9966102	best: 0.9966102 (2)	total: 1.32s	remaining: 7m 20s
3:	learn: 0.9999948	test: 0.9967351	best: 0.9967351 (3)	total: 1.74s	remaining: 7m 13s
4:	learn: 0.9999948	test: 0.9967870	best: 0.9967870 (4)	total: 2.15s	remaining: 7m 8s
5:	learn: 0.9999948	test: 0.9968519	best: 0.9968519 (5)	total: 2.61s	remaining: 7m 12s
6:	learn: 0.9999948	test: 0.9968626	best: 0.9968626 (6)	total: 3.05s	remaining: 7m 12s
7:	learn: 0.9999948	test: 0.9969123	best: 0.9969123 (7)	total: 3.52s	remaining: 7m 17s
8:	learn: 0.9999948	test: 0.9969063	best: 0.9969123 (7)	total: 4s	remaining: 7m 21s
9:	learn: 0.9999948	test: 0.9969496	best: 0.9969496 (9)	total: 4.5s	remaining: 7m 25s
10:	learn: 0.9999948	test: 0.996952